In [1]:
import pandas as pd
from elasticsearch import Elasticsearch
import elasticsearch.helpers
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import numpy as np

es = Elasticsearch('elasticsearch', port=9200)

body={"query": {"match_all": {}}}

In [2]:
results = elasticsearch.helpers.scan(es, query=body, index="gitter_enriched_raw")
df_gitter = pd.DataFrame.from_dict([document['_source'] for document in results])
df_gitter.grimoire_creation_date = pd.to_datetime(df_gitter.grimoire_creation_date) # time that message was sent
df_gitter = df_gitter.sort_values(by='grimoire_creation_date')

results = elasticsearch.helpers.scan(es, query=body, index="github2-issues_enriched")
df_github_issues = pd.DataFrame.from_dict([document['_source'] for document in results])
df_github_issues.grimoire_creation_date = pd.to_datetime(df_github_issues.grimoire_creation_date) # time that comment was created

In [3]:
gitter_issues = {}
change_ratio_rows = []

for i,g in df_gitter.groupby('project'):
    gitter_issues[i] = dict()
    for i2, g2 in g[g.issues.astype(bool)].iterrows():
        for item in g2.issues:
            if 'repo_type' in item.keys() and 'number' in item.keys():
                if item['repo_type'] == 'project':                        
                    change_ratio_rows.append({
                        'project' : i,
                        'gitter_message_date': g2.grimoire_creation_date,
                        'gitter_message_uuid':  g2.uuid,
                        'issue_number': item['number'],
                        'min_date': g2.grimoire_creation_date - timedelta(days=7),
                        'original_date': g2.grimoire_creation_date,
                        'max_date': g2.grimoire_creation_date + timedelta(days=7),
                        'comments_after': len(df_github_issues[(df_github_issues['is_github_issue_comment'] == 1) 
                                           & (df_github_issues.grimoire_creation_date <= g2.grimoire_creation_date + timedelta(days=7)) 
                                           & (df_github_issues.grimoire_creation_date > g2.grimoire_creation_date) 
                                           & (df_github_issues['project'] == i) & (df_github_issues['issue_id_in_repo'] == item['number']) ]),
                        'comments_before': len(df_github_issues[(df_github_issues['is_github_issue_comment'] == 1) 
                                           & (df_github_issues.grimoire_creation_date <= g2.grimoire_creation_date) 
                                           & (df_github_issues.grimoire_creation_date >= g2.grimoire_creation_date - timedelta(days=7)) 
                                           & (df_github_issues['project'] == i) & (df_github_issues['issue_id_in_repo'] == item['number']) ]),
                        'total_comments': len(df_github_issues[(df_github_issues['is_github_issue_comment'] == 1) & (df_github_issues.grimoire_creation_date <= g2.grimoire_creation_date + timedelta(days=7)) & 
                                          (df_github_issues.grimoire_creation_date >= g2.grimoire_creation_date - timedelta(days=7)) & (df_github_issues['project'] == i) & (df_github_issues['issue_id_in_repo'] == item['number']) ])
                    })

In [4]:
gitter_comments_count_1_week = pd.DataFrame(change_ratio_rows)
gitter_comments_count_1_week['ratio'] = gitter_comments_count_1_week['comments_after'] / gitter_comments_count_1_week['comments_before']

In [6]:
gitter_comments_count_1_week.to_csv('gitter_comments_count_1_week.csv', index=False)